<a href="https://colab.research.google.com/github/deepskyai/deepskyai.github.io/blob/main/rosterparse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Requirements
Run them once and then click on Restart Runtime

---

→ → → Options Menu at the top of the Page\
→ → Runtime\
→ Restart Runtime



In [ ]:
!pip install tabula-py
!pip install camelot-py
!sudo apt-get update
!sudo apt-get install build-essential libpoppler-cpp-dev pkg-config python-dev
!pip install pdftotext
!pip install pdf2image
!apt-get install poppler-utils 
!pip install Pillow==6.2.2
!pip install dataframe-image
!pip install pdfplumber

# **MAIN CODE**

---

***Go to the bottom of the page to insert PDF***


In [ ]:
%matplotlib inline

import tempfile
from io import StringIO
import tabula
import camelot
import pandas as pd
import numpy as np
import pdftotext
from PIL import Image, ImageFont, ImageDraw
from pdf2image import convert_from_path
import pdfplumber
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import dataframe_image as dfi
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os 

################################################################################    GOOGLE DRIVE OPTION

#from google.colab import drive
#drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)
#os.chdir("/content/drive/MyDrive/dataframe_job/Main/")
#print()

################################################################################    GOOGLE DRIVE OPTION

def make_df_image(table,
                  max_cols=-1,
                  max_rows=-1):
    """Return dataframe as image."""
    with tempfile.NamedTemporaryFile(suffix='.jpg',
                                     delete=False) as tmp:
        dfi.export(table, tmp.name, max_cols=max_cols, max_rows=max_rows)
        image = mpimg.imread(tmp.name)
        return image

def make_lines_image(lines,
                     mrgn=15,
                     background=(255, 255, 255),
                     text_color=(0, 0, 0),
                     font_size=8):
    """Return raw text in lines as image."""
    lines = pd.Series(lines)
    longest_line = lines[lines.str.len().idxmax()]
    image = Image.new("RGBA", (1, 1))
    draw = ImageDraw.Draw(image)
    monospace = ImageFont.truetype(r'С:\Windows\Fonts\cour.ttf', font_size)
    line_width, line_height = draw.textsize(longest_line, monospace)
    img_width, img_height = (line_width + mrgn * 2,
                             len(lines) * line_height + mrgn * 2)
    image = Image.new("RGBA", (img_width, img_height), background)
    draw = ImageDraw.Draw(image)
    x, y0 = (mrgn, mrgn)
    for n, line in enumerate(lines):
        y = y0 + n * line_height
        draw.text((x, y), line, text_color, monospace)
    return image


################################################################################
import pdfplumber

def Checker(pdf):  
  ap = pdf

  all_pages = []

  with pdfplumber.open(ap) as pdf:
    i = 0
    while i >= 0:
      #print(i)
      page = pdf.pages[i]
      text = page.extract_text()
      dt = text.rsplit("\n")
      
      pages = dt[1]
      last_page = pages[10:]
      #print("last page is: ", last_page)

      if i == (int(last_page)-1):
        i = -1
      else:
        i += 1

      all_pages.append(dt)

  i = 0
  for pages in all_pages:
    #print()
    del pages[0:5]
    test = pages[0]
    if test == "Crew onboard":
      break
    i += 1

  i = i-1

  return i




def getter(ap):
  all_pages = []

  with pdfplumber.open(ap) as pdf:
    i = 0
    while i >= 0:
      #print(i)
      page = pdf.pages[i]
      text = page.extract_text()
      dt = text.rsplit("\n")
      
      pages = dt[1]
      last_page = pages[10:]
      #print("last page is: ", last_page)

      if i == (int(last_page)-1):
        i = -1
      else:
        i += 1

      all_pages.append(dt)

  # this is main list with all pages appended as lists in it
  #print(all_pages)

  import re


  max = len(all_pages)

  biggest = []
  max_count = 0

  for pages in all_pages:
    max_count += 1

    new_page = pages

    month_year = new_page[2]  
    #print(month_year)
    my = month_year.replace(" - ", "___")
    my = my.split('___')
    my1 = my[0]
    my2 = my[1]

    #print(my1)
    home_base = my1[:3]
    #print(home_base)

    start_month = my1[7:]
    #print("final ", start_month)

    end_month = my2[:7]
    #print("final ", end_month)

    start_month = start_month[:-2]
    #start_month = start_month[-3:]     # date
    #print(start_month)

    end_month = end_month[:-2]
    #end_month = end_month[-3:]       # date
    #print(end_month)


    ####################################################

    month_year = month_year.replace(" ", "")
    month_year = month_year[6:]
    timescale = month_year.rsplit('-', 1)
    timescale = timescale[0]

    month_year = timescale[2:]
    #print(month_year)
    s = len(month_year)
    global_month = month_year[:-2]
    global_year = month_year[-2:]

    #print("----------------------------------------------------------------------")
    #print(global_month)
    #print(global_year)
    #print("----------------------------------------------------------------------")

    #global_month = start_month + "-" + end_month


  #print(global_month)
  #print(global_year)
  #print(max)
  return global_month, global_year, max, home_base



def DataFrame_Function(PDF, i):

  pd.set_option('display.max_colwidth', None)

  pdf_img = convert_from_path(PDF, dpi=300)[0]
  
  dataframe = (camelot.read_pdf(PDF,flavor="stream",suppress_stdout=True,pages="all")[i].df)

  return dataframe




################################################################################



def Main(pdf, i):

  PDF = pdf

  ap = PDF
  global_month = ''
  global_year = ''

  dataframes = DataFrame_Function(PDF, i)

  df = dataframes

  return df


################################################################################################################################################################


def col_con1(df, index): # Date\nDD
  global final_date
  date = []
  dd = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      day = item[-3:]
      #print(day)
      if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
        date.append(item)
        dd.append("")
      else:
        new = date[-1]
        date.append(new)
        dd.append(item)

    elif len(item) == 0:
      try:
        new = date[-1]
        date.append(new)
        dd.append("")
      except:
        date.append(final_date)
        dd.append("")

    elif (len(item) >= 1) and (len(item) <= 4):
      new = date[-1]
      date.append(new)
      dd.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          day = items[-3:]
          if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
            date.append(items)
          else:
            dd.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          dd.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(date))
  #print(date)
  #print(len(dd))
  #print(dd) 

  final_date = date[-1]

  return date, dd

#############################################

def col_con2(df, index): # C/I\nOrig
  time = []
  cc = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      time.append(item)
      cc.append("")

    elif len(item) == 0:
      time.append("")
      cc.append("")

    elif len(item) == 3:
      time.append("")
      cc.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          time.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          cc.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(time))
  #print(time)
  #print(len(cc))
  #print(cc) 

  return time, cc


#############################################

def col_con3(df, index): # C/O\nAC
  time = []
  ac = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      time.append(item)
      ac.append("")

    elif len(item) == 0:
      time.append("")
      ac.append("")

    elif len(item) == 3:
      time.append("")
      ac.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          time.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          ac.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(time))
  #print(time)
  #print(len(ac))
  #print(ac) 

  return time, ac


#############################################

def col_con4(df, index): # ATA\nC/O
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return ata, co

#############################################
global final_date

def col_con5(df, index): # Date
  global final_date
  date = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      date.append(item)
      
    else:
      try:
        new = date[-1]
        date.append(new)
      except:
        date.append(final_date)
        

  #print(col_one_list)
  #print(len(date))
  #print(date)
  
  final_date = date[-1]

  return date



#############################################

def col_con6(df, index): # BLHCR\nFDUTY
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return co, ata 



#############################################

def col_con7(df, index): # AC\nRq
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 3:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return co, ata 



#############################################

def col_con8(df, index): # C/I\nDate\nDD\nActivity\nOrig
  global final_date
  
  cc = 'ASP ABH ARY ARM AVV AYQ BSJ BNK BZD BCI BHS BRT BEU BLN BKQ BLT BQL BRK BWQ BNE BHQ BME BDB CNS CDU CBR CVQ CED CTL CCL CMQ CVC CNJ CAZ CUQ CFS CPD CTN OOM COJ CNB CMD CWW CYG CWT CMA DRW DNQ DRB DPO DRN DBO DYA ECH ELC EMD EPR FIZ FRB FOS GPN GET GLT GLI OOL GUL GOV GFN GFF GTE GUH HCQ HLT HTI HXX HVB HBA HTU HID HSM IVR KGI KTA KPS KRA KNS KGY KGC KWM KNX TGN LST LEA LGH LER LNO LHG LSY IRG LRE MKY MNG MRG MKR MEB MEL MBW MIM MQL MOV MRZ MYA MGB MHU ISA WME MMG DGE NAA NRA QRM NTL ZNE RPM NTN NOA OKY OAG PBO PKE PER JAD PUG PHE PKT PLO PQQ PPI PTJ PPP ULP YAMB DCN YMES XRH RMK ROK RMA MJK SHT SIO SNB SWC MCY SWH BWU SYD TMW TRO TEF TEM TCA THG XTG TYB KTR WTB TSV TUM WGA WGE WGT WKB QRR WEI SXE WWY WYA WUN WNR WIN WOL BWT NGA'
  city_codes = cc.split(' ')
  
  ci = []
  date = []
  dd = []  
  activity = []
  orig = []

  ci_n = ""
  date_n = ""
  dd_n = ""
  activity_n = ""
  orig_n = ""


  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    item = item.split()

    for items in item:
      if len(items) == 5:
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          date_n = items
          
      if (len(items) == 5) and (":" in items):
        ci_n = items

      if items in city_codes:    
        orig_n = items        

      if (len(items) >= 3) and (len(items) <= 5) and (":" not in items) and (items not in city_codes):
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          a = 0
        else:
          activity_n = items
          #dd_n = ""

      if (len(items) < 3):
        dd_n = items
    
    #if len(items) == 5:
    date.append(date_n)
    ci.append(ci_n)
    orig.append(orig_n)
    activity.append(activity_n)
    try:
      dd.append(dd_n)
    except:
      dd.append("")


    ci_n = ""
    dd_n = ""
    activity_n = ""
    orig_n = ""


  #print(ci)
  #print(len(ci))
  #print(date)
  #print(len(date))
  #print(dd)
  #print(len(dd))
  #print(activity)
  #print(len(activity))
  #print(orig)
  #print(len(orig))

  return ci, date, dd, activity, orig


#############################################

def col_con9(df, index): # C/I\nDate\nDD\nActivity
  global final_date

  ci = []
  date = []
  dd = []  
  activity = []

  ci_n = ""
  date_n = ""
  dd_n = ""
  activity_n = ""


  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    item = item.split()

    for items in item:
      if len(items) == 5:
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          date_n = items
          
      if (len(items) == 5) and (":" in items):
        ci_n = items
      

      if (len(items) >= 3) and (len(items) <= 5) and (":" not in items):
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          a = 0
        else:
          activity_n = items
          #dd_n = ""

      if (len(items) < 3):
        dd_n = items
        
    #if len(items) == 5:
    date.append(date_n)
    ci.append(ci_n)
    activity.append(activity_n)
    try:
      dd.append(dd_n)
    except:
      dd.append("")


    ci_n = ""
    dd_n = ""
    activity_n = ""


  #print(ci)
  #print(len(ci))
  #print(date)
  #print(len(date))
  #print(dd)
  #print(len(dd))
  #print(activity)
  #print(len(activity))
  #print(orig)
  #print(len(orig))

  return ci, date, dd, activity


#############################################

def col_con10(df, index): # C/O\nAC\nRq
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:

    if (len(item) < 5) and (len(item) > 0):
      time.append("")
      cc.append(item)

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5) and (":" in item):
      time.append(item)
      cc.append("")

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if len(items) == 3:    
          cc.append(items)        

        if (":" in items):
          time.append(items)


  return time, cc


#############################################

def col_con11(df, index): # Orig
  ccs = 'ASP ABH ARY ARM AVV AYQ BSJ BNK BZD BCI BHS BRT BEU BLN BKQ BLT BQL BRK BWQ BNE BHQ BME BDB CNS CDU CBR CVQ CED CTL CCL CMQ CVC CNJ CAZ CUQ CFS CPD CTN OOM COJ CNB CMD CWW CYG CWT CMA DRW DNQ DRB DPO DRN DBO DYA ECH ELC EMD EPR FIZ FRB FOS GPN GET GLT GLI OOL GUL GOV GFN GFF GTE GUH HKT HCQ HLT HTI HXX HVB HBA HTU HID HSM IVR KGI KTA KPS KRA KNS KGY KGC KWM KNX TGN LST LEA LGH LER LNO LHG LSY IRG LRE MKY MNG MRG MKR MEB MEL MBW MIM MQL MOV MRZ MYA MGB MHU ISA WME MMG DGE NAA NRA QRM NTL ZNE RPM NTN NOA OKY OAG PBO PKE PER JAD PUG PHE PKT PLO PQQ PPI PTJ PPP ULP YAMB DCN YMES XRH RMK ROK RMA MJK SHT SIO SNB SWC MCY SWH BWU SYD TMW TRO TEF TEM TCA THG XTG TYB KTR WTB TSV TUM WGA WGE WGT WKB QRR WEI SXE WWY WYA WUN WNR WIN WOL BWT NGA'
  city_codes = ccs.split(' ')
  
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
  
    if (len(item) == 3):
      time.append(item)
      cc.append("")

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5):
      time.append("")
      cc.append(item)

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if items in city_codes:    
          time.append(items)        

        if (":" in items):
          cc.append(items)


  return time, cc



#############################################

def col_con12(df, index): # C/O\nAC\nRq
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    
    if (len(item) < 5) and (len(item) > 0):
      time.append("")
      cc.append(item)

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5) and (":" in item):
      time.append(item)
      cc.append("")

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if len(items) == 3:    
          cc.append(items)        

        if (":" in items):
          time.append(items)



  return time, cc

################################################################################################################################################################


def Final(df):
  df1 = df
  column_headers = df.columns.values.tolist()
  #print(column_headers)
  #print()

  cols_index = []

  i = 0
  for columns in column_headers:
    if columns == "Date\nDD":
      index = i
      date, dd = col_con1(df1, column_headers[i])
      
      cols_index.append(index)    
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Date", date)
      df1.insert(((index+1)+1), "Duty Designator", dd)
    
    if columns == "C/I\nActivity":
      index = i
      date, dd = col_con2(df1, (column_headers[i+1]))

      df1.rename(columns = {'C/I\nActivity':'Activity'}, inplace = True)
      cols_index.append(index)
      columns = column_headers[index+1]
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", date)
      df1.insert(((index+1)+1), "Origin", dd)
      
    if columns == "C/I\nOrig":
      index = i
      date, dd = col_con2(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", date)
      df1.insert(((index+1)+1), "Origin", dd)
    
    if columns == "ATA\nC/O":
      index = i
      date, dd = col_con3(df1, (column_headers[i+1]))
      ata, co = col_con4(df1, (column_headers[i]))

      cols_index.append(index)
      columns1 = column_headers[index+1]
      df1.drop(columns1, axis=1, inplace=True)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)
      
      df1.insert(((index+1)+2), "ATA", ata)
      df1.insert(((index+1)+3), "C O", co)
    
      df1["Check Out Time"] = df1["Check Out Time"] + df1["C O"]
      df1.drop('C O', axis=1, inplace=True)


    if columns == "STA\nC/O":
      index = i
      date, dd = col_con3(df1, (column_headers[i+1]))
      ata, co = col_con4(df1, (column_headers[i]))

      cols_index.append(index)
      columns1 = column_headers[index+1]
      df1.drop(columns1, axis=1, inplace=True)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)
      
      df1.insert(((index+1)+2), "STA", ata)
      df1.insert(((index+1)+3), "C O", co)
    
      df1["Check Out Time"] = df1["Check Out Time"] + df1["C O"]
      df1.drop('C O', axis=1, inplace=True)



    if columns == "BLHCR\nFDUTY":
      index = i
      blhcr, fduty = col_con6(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      #df1.insert((index+1), "BLHCR", blhcr)
      #df1.insert(((index+1)+1), "FDUTY", fduty)

    if columns == "AC\nRq":
      index = i
      blhcr, fduty = col_con7(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "AC", fduty)
      df1.insert(((index+1)+1), "Rq", blhcr)

    if columns == "C/I\nDate\nDD\nActivity\nOrig":
      index = i
      ci, date, dd, activity, orig = col_con8(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", ci)
      df1.insert(((index+1)+1), "Date", date)
      df1.insert(((index+1)+1), "DD", dd)
      df1.insert(((index+1)+1), "Activity", activity)
      df1.insert(((index+1)+1), "Orig", orig)

    if columns == "C/I\nDate\nDD\nActivity":
      index = i
      ci, date, dd, activity = col_con9(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", ci)
      df1.insert(((index+1)+1), "Date", date)
      df1.insert(((index+1)+1), "DD", dd)
      df1.insert(((index+1)+1), "Activity", activity)

    if columns == "C/O\nAC\nRq":
      index = i
      date, dd = col_con10(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)

    if (columns == "Orig"):
      index = i
      try:
        date, dd = col_con11(df1, (column_headers[i]))


        cols_index.append(index)
        df1.drop(columns, axis=1, inplace=True)
        
        df1.insert((index+1), "Origin", date)
        df1.insert(((index+1)+1), "C I", dd)

        df1["Check In Time"] = df1["Check In Time"] + df1["C I"]
        df1.drop('C I', axis=1, inplace=True)

      except:
        pass

    if columns == "ATA\nDest":
      index = i
      date, dd = col_con12(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "ATA", date)
      df1.insert(((index+1)+1), "Dest", dd)

    if columns == "STA\nDest":
      index = i
      date, dd = col_con12(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "STA", date)
      df1.insert(((index+1)+1), "Dest", dd)

    if columns == "Date":
      index = i
      date = col_con5(df1, column_headers[i])
      
      cols_index.append(index)    
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "nDate", date)
    
      df1["Date"] = df1["nDate"]
      df1.drop("nDate", axis=1, inplace=True)


    i += 1

  try:
    df1.drop("BLH", axis=1, inplace=True)
    df1.drop("Rq", axis=1, inplace=True)
    df1.drop("BLHCR", axis=1, inplace=True)
    df1.drop("FDUTY", axis=1, inplace=True)
  except:
    pass

  return df1



################################################################################################################################################################



def Last(pdf, df):

  global_month = ''
  global_year = ''
  max = 0

  global_month, global_year, max, home_base = getter(pdf)

  #df = df

  df['WDO'] = 'FALSE'

  arr = []

  #print(df)
  try:
    df.loc[df['Activity'].str.startswith('WDO'), 'WDO'] = 'TRUE'
  except:
    df2=df.dropna()
    #print(df2)

  lister = df.to_dict('dict')


  data = lister['WDO']


  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows = []
  value = 'TRUE'
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Date']

  rows1 = []
  dates = []

  for item in rows:  
    value_at_index = list(data2.values())[item]
    #print(value_at_index)
    dates.append(value_at_index)


  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows1.append(key)
              do_exist = True
      return do_exist


  for value in dates:
    # Iterate over all key, value pairs in dict and check if value exist
    if check_value_exist(data2, value, rows1):
        o = 0
        #print(f"Yes, Value: '{value}' exists in dictionary")
    else:
        o = 0
        #print(f"No, Value: '{value}' does not exists in dictionary")

  def checkKey(data, row):
    data[row] = 'TRUE'  
    #lister['WDO'][row].update('TRUE')
    #lister['WDO'][row].pop(1)
    #print(data)
          
  for row in rows1:
    checkKey(data, row)

  lister['WDO'] = data

  new = pd.DataFrame.from_dict(lister)
  #print(new)
  #new = new.iloc[: , 1:]
  #new['WDO'] = new['WDO'] 
  #new['WDO?'] = new.loc[:, 'WDO']

  def date_cols(df):
    date = []
    day = []

    col_one_list = df['Date'].tolist()
    
    res = []

    for item in col_one_list:
      big = item
      small = item
      date1 = big[:-3]
      day1 = small[2:]
      date.append(date1)
      day.append(day1)        

    

    #print(date)
    #print(len(date))
    #print()
    #print(dd)
    #print(len(dd))

    #print(col_one_list)
    
    return date, day
    
  
  column_headers1 = list(new.columns)
  #print(column_headers1)
  if ("Aircraft Type" in column_headers1) and ("AC" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Aircraft Type"] = new["AC"] + '' + new["Aircraft Type"]
    new.drop('AC', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Duty Designator" in column_headers1) and ("DD" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Duty Designator"] = new["DD"] + '' + new["Duty Designator"]
    new.drop('DD', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Check Out Time" in column_headers1) and ("C/O" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Check Out Time"] = new["C/O"] + '' + new["Check Out Time"]
    new.drop('C/O', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Check In Time" in column_headers1) and ("C/I" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Check In Time"] = new["C/I"] + '' + new["Check In Time"]
    new.drop('C/I', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Orig" in column_headers1) and ("Origin" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Origin"] = new["Orig"] + '' + new["Origin"]
    new.drop('Orig', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0


  column_headers = list(new.columns)
  #print(column_headers)

  # Activity', 'Date', 'Duty Designator', 'C/I', 'Orig', 'ATD', 'Dest', 'ATA', 'C/O', 'AC', 'DD', 'WDO
  for names in column_headers:
    if names == "C/I":
      new.rename(columns = {'C/I':'Check In Time'}, inplace = True)
    elif names == "Orig":
      new.rename(columns = {'Orig':'Origin'}, inplace = True)
    elif names == "C/O":
      new.rename(columns = {'C/O':'Check Out Time'}, inplace = True)
    elif names == "AC":
      new.rename(columns = {'AC':'Aircraft Type'}, inplace = True)    
    elif names == "DD":
        new.rename(columns = {'DD':'Duty Designator'}, inplace = True)
        
  #print(new)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'ATA', 'Check Out', 'Date  ==>   better names
    dataframe = new[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Origin', 'WDO', 'ATD', 'Dest', 'ATA', 'Check Out Time', 'Date']]
  except:
    dataframe = new[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Origin', 'WDO', 'STD', 'Dest', 'STA', 'Check Out Time', 'Date']]


  date = []
  day = []

  date, day = date_cols(dataframe)

  dataframe.rename(columns = {'Date':'full date'}, inplace = True)
  #print()
  #print("FINAL")

  dataframe.insert(1, "Date", date)
  dataframe.insert(2, "Day", day)
  dataframe.insert(3, "Month", global_month)
  dataframe.insert(4, "Year", global_year)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Destination', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out', 'Date', 'Day', 'Month', 'Year'
    #dataframe1 = dataframe[['Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'WDO', 'ATA', 'Check Out']]
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']]
  except:
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'STD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'STA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']]


  #print(dataframe)  

  return dataframe

#################################################################################



def Neat(dataframe):

  dataframe['Origin'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['Origin'].transform(lambda x: ''.join(x))
  dataframe['Dest'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['Dest'].transform(lambda x: ''.join(x))

  try:
    dataframe['ATA'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['ATA'].transform(lambda x: ''.join(x))
    dataframe['ATD'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['ATD'].transform(lambda x: ''.join(x))
  except:
    dataframe['STA'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['STA'].transform(lambda x: ''.join(x))
    dataframe['STD'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['STD'].transform(lambda x: ''.join(x))  


  def add_one(x):
    #print(x)
    x = str(x)
    if len(x) > 3:
      x = x[:3]
    else:
      x = x
    
    return x

  def add_two(x):
    #print(x)
    x = str(x)
    if len(x) > 5:
      x = x[:5]
    else:
      x = x
    
    return x


  dataframe['Origin'] = dataframe['Origin'].apply(add_one)
  dataframe['Dest'] = dataframe['Origin'].apply(add_one)

  try:
    dataframe['ATA'] = dataframe['ATA'].apply(add_two)
    dataframe['ATD'] = dataframe['ATD'].apply(add_two)
  except:  
    dataframe['STA'] = dataframe['STA'].apply(add_two)
    dataframe['STD'] = dataframe['STD'].apply(add_two)



################################################################################################################################################################


def lines(pdf, df):

  global_month = ''
  global_year = ''
  max = 0

  global_month, global_year, max, home_base = getter(pdf)

  lister = df.to_dict('dict')


  data = lister['Dest']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Dest']

  dates = []

  for item in rows1:
    value_at_index = list(data2.values())[item+1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  #print("data from dest    ", data)
  lister['Dest'].update(data)

  try:
    data = lister['ATA']
  except:
    data = lister['STA']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  try:
    data2 = lister['ATA']
  except:
    data2 = lister['STA']

  dates = []

  for item in rows1:  
    value_at_index = list(data2.values())[item+1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  
  try:
    #print("data from ata    ", data)
    lister['ATA'].update(data)
  except:
    #print("data from sta    ", data)
    lister['STA'].update(data)

  save1 = rows1

  ##########################################################################################################################################################


  data = lister['Origin']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Origin']

  dates = []

  for item in rows1:
    value_at_index = list(data2.values())[item-1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  #print("data from origin    ", data)
  lister['Origin'].update(data)


  save2 = rows1

  try:
    data = lister['ATD']
  except:
    data = lister['STD']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  try:
    data2 = lister['ATD']
  except:
    data2 = lister['STD']

  dates = []

  _o0 = lister['Origin']
  try:
    _o1 = lister['ATD']
  except:
    _o1 = lister['STD']

  _o2 = lister['Dest']
  try:
    _o3 = lister['ATA']
  except:
    _o3 = lister['STA']

  for item in rows1:  
    if (list(_o0.values())[item] == "") and (list(_o1.values())[item] == ""):
      value_at_index = list(data2.values())[item-1]
      #print(value_at_index)
      dates.append(value_at_index)
    else:
      value_at_index = list(data2.values())[item+1]
      #print(value_at_index)
      dates.append(value_at_index)


  def checkKey(data, row, dates):
    data[row] = dates
  
  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  try:
    #print("data from atd    ", data)
    lister['ATD'].update(data)
  except:
    #print("data from std    ", data)
    lister['STD'].update(data)

  #print(lister)

  #save2 = rows1

  ##########################################################################################################################################################
  
  new = []
  lol = ''
  i = 0 
  for dates in save1:
    lol = str(save1[i]) + "," + str(save2[i]) 
    i += 1
    new.append(lol)

  #print(new)
  #print()

  #######################
  
  data1 = lister['Date_3']
  data2 = lister['Date_4']
  
  def checkKey(data, row, dates):
    data[row] = dates

  for some in new:
    one, two = some.split(',')
    one = int(one)
    two = int(two)
    #print(one)
    #print(two)

    checkKey(data1, one, data1[two+1])
    checkKey(data2, one, data2[two+1])
    

  #######################
  

  data1 = lister['Day_3']
  data2 = lister['Day_4']
  data3 = lister['Check Out Time']
  
  def checkKey(data, row, dates):
    data[row] = dates
    if dates != "":
      #print("finally-------------------------------------------------------------------------------------")
      data[row] = str(dates)
    
  for some in new:
    one, two = some.split(',')
    one = int(one)
    two = int(two)
    #print(one)
    #print(two)
    checkKey(data1, one, data1[two+1])
    checkKey(data2, one, data2[two+1])
    checkKey(data3, one, data3[two])
    
  ##########################################################################################################################################################

  dataframe = pd.DataFrame.from_dict(lister)

  #print(save2)
  #print()
  for i in save2:
    #print(i)
    dataframe = dataframe.drop(i)

  dataframe = dataframe.reset_index(drop=True)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Destination', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out', 'Date', 'Day', 'Month', 'Year'
    #dataframe1 = dataframe[['Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'WDO', 'ATA', 'Check Out']]
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date_2', 'Day_2', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date_3', 'Day_3', 'Month', 'Year', 'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]
  except:
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Origin', 'WDO', 'STD', 'Date_2', 'Day_2', 'Month', 'Year', 'Dest', 'WDO', 'STA', 'Date_3', 'Day_3', 'Month', 'Year', 'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]


  #print(dataframe)  

  return dataframe

#################################################################################






################################################################################################################################################################

import PyPDF2


### DRIVER CODE


### FILE PATH
##################################                          INSERT FILE NAME UNDER HERE ↓↓↓↓                          ##################################
pdf = 'Sep Original Roster.pdf'
##################################                          INSERT FILE NAME UNDER HERE ↑↑↑↑                          ##################################

pages = Checker(pdf)
print("total pages: ", (pages+1))

appended_df = []

#for i in range(1):
for i in range((pages+1)):
  #print(i)
  #i = 1
  df = Main(pdf, i)
  df = pd.DataFrame(df)
  df = df.rename(columns=df.iloc[1]).loc[2:]
  df = Final(df)
  appended_df.append(df)


import pandas as pd
try:
  df = pd.concat(appended_df, ignore_index=True)
except:
  new_appended_df = appended_df[:-1]
  df = pd.concat(new_appended_df, ignore_index=True)

#df

df = df.reset_index(drop=True)

dataframe = Last(pdf, df)
dataframe = dataframe.fillna('')


#Neat(dataframe)

cols = []
count = 1
for column in dataframe.columns:
    if column == 'Date':
        cols.append(f'Date_{count}')
        count+=1
        continue
    cols.append(column)
dataframe.columns = cols


cols = []
count = 1
for column in dataframe.columns:
    if column == 'Day':
        cols.append(f'Day_{count}')
        count+=1
        continue
    cols.append(column)
dataframe.columns = cols


dataframe = lines(pdf, dataframe)

print()
dataframe

**INSERT FILE NAME WITH EXTENSION AND RUN THE CELL.**

---

*(You can import files in this notebook by clicking on the document icon on the left taskbar and dragging and dropping your pdf there)*

---

Click on the wand icon once execution is finished for a more aesthetic display

# To Generate CSV File from DataFrame

In [ ]:
dataframe.to_csv('CSV_File.csv')

# New Section (Under Testing)

In [ ]:
%matplotlib inline

import tempfile
from io import StringIO
import tabula
import camelot
import pandas as pd
import numpy as np
import pdftotext
from PIL import Image, ImageFont, ImageDraw
from pdf2image import convert_from_path
import pdfplumber
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import dataframe_image as dfi
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import os 

################################################################################    GOOGLE DRIVE OPTION

#from google.colab import drive
#drive.mount('/content/drive')
#drive.mount('/content/drive', force_remount=True)
#os.chdir("/content/drive/MyDrive/dataframe_job/Main/")
#print()

################################################################################    GOOGLE DRIVE OPTION

def make_df_image(table,
                  max_cols=-1,
                  max_rows=-1):
    """Return dataframe as image."""
    with tempfile.NamedTemporaryFile(suffix='.jpg',
                                     delete=False) as tmp:
        dfi.export(table, tmp.name, max_cols=max_cols, max_rows=max_rows)
        image = mpimg.imread(tmp.name)
        return image

def make_lines_image(lines,
                     mrgn=15,
                     background=(255, 255, 255),
                     text_color=(0, 0, 0),
                     font_size=8):
    """Return raw text in lines as image."""
    lines = pd.Series(lines)
    longest_line = lines[lines.str.len().idxmax()]
    image = Image.new("RGBA", (1, 1))
    draw = ImageDraw.Draw(image)
    monospace = ImageFont.truetype(r'С:\Windows\Fonts\cour.ttf', font_size)
    line_width, line_height = draw.textsize(longest_line, monospace)
    img_width, img_height = (line_width + mrgn * 2,
                             len(lines) * line_height + mrgn * 2)
    image = Image.new("RGBA", (img_width, img_height), background)
    draw = ImageDraw.Draw(image)
    x, y0 = (mrgn, mrgn)
    for n, line in enumerate(lines):
        y = y0 + n * line_height
        draw.text((x, y), line, text_color, monospace)
    return image


################################################################################
import pdfplumber

def Checker(pdf):  
  ap = pdf

  all_pages = []

  with pdfplumber.open(ap) as pdf:
    i = 0
    while i >= 0:
      #print(i)
      page = pdf.pages[i]
      text = page.extract_text()
      dt = text.rsplit("\n")
      
      pages = dt[1]
      last_page = pages[10:]
      #print("last page is: ", last_page)

      if i == (int(last_page)-1):
        i = -1
      else:
        i += 1

      all_pages.append(dt)

  i = 0
  for pages in all_pages:
    #print()
    del pages[0:5]
    test = pages[0]
    if test == "Crew onboard":
      break
    i += 1

  i = i-1

  return i




def getter(ap):
  all_pages = []

  with pdfplumber.open(ap) as pdf:
    i = 0
    while i >= 0:
      #print(i)
      page = pdf.pages[i]
      text = page.extract_text()
      dt = text.rsplit("\n")
      
      pages = dt[1]
      last_page = pages[10:]
      #print("last page is: ", last_page)

      if i == (int(last_page)-1):
        i = -1
      else:
        i += 1

      all_pages.append(dt)

  # this is main list with all pages appended as lists in it
  #print(all_pages)

  import re


  max = len(all_pages)

  biggest = []
  max_count = 0

  for pages in all_pages:
    max_count += 1

    new_page = pages

    month_year = new_page[2]  
    #print(month_year)
    my = month_year.replace(" - ", "___")
    my = my.split('___')
    my1 = my[0]
    my2 = my[1]

    #print(my1)
    home_base = my1[:3]
    #print(home_base)

    start_month = my1[7:]
    #print("final ", start_month)

    end_month = my2[:7]
    #print("final ", end_month)

    start_month = start_month[:-2]
    #start_month = start_month[-3:]     # date
    #print(start_month)

    end_month = end_month[:-2]
    #end_month = end_month[-3:]       # date
    #print(end_month)


    ####################################################

    month_year = month_year.replace(" ", "")
    month_year = month_year[6:]
    timescale = month_year.rsplit('-', 1)
    timescale = timescale[0]

    month_year = timescale[2:]
    #print(month_year)
    s = len(month_year)
    global_month = month_year[:-2]
    global_year = month_year[-2:]

    #print("----------------------------------------------------------------------")
    #print(global_month)
    #print(global_year)
    #print("----------------------------------------------------------------------")

    #global_month = start_month + "-" + end_month


  #print(global_month)
  #print(global_year)
  #print(max)
  return global_month, global_year, max, home_base



def DataFrame_Function(PDF, i):

  pd.set_option('display.max_colwidth', None)

  pdf_img = convert_from_path(PDF, dpi=300)[0]
  
  dataframe = (camelot.read_pdf(PDF,flavor="stream",suppress_stdout=True,pages="all")[i].df)

  return dataframe




################################################################################



def Main(pdf, i):

  PDF = pdf

  ap = PDF
  global_month = ''
  global_year = ''

  dataframes = DataFrame_Function(PDF, i)

  df = dataframes

  return df


################################################################################################################################################################


def col_con1(df, index): # Date\nDD
  global final_date
  date = []
  dd = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      day = item[-3:]
      #print(day)
      if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
        date.append(item)
        dd.append("")
      else:
        new = date[-1]
        date.append(new)
        dd.append(item)

    elif len(item) == 0:
      try:
        new = date[-1]
        date.append(new)
        dd.append("")
      except:
        date.append(final_date)
        dd.append("")

    elif (len(item) >= 1) and (len(item) <= 4):
      new = date[-1]
      date.append(new)
      dd.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          day = items[-3:]
          if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
            date.append(items)
          else:
            dd.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          dd.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(date))
  #print(date)
  #print(len(dd))
  #print(dd) 

  final_date = date[-1]

  return date, dd

#############################################

def col_con2(df, index): # C/I\nOrig
  time = []
  cc = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      time.append(item)
      cc.append("")

    elif len(item) == 0:
      time.append("")
      cc.append("")

    elif len(item) == 3:
      time.append("")
      cc.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          time.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          cc.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(time))
  #print(time)
  #print(len(cc))
  #print(cc) 

  return time, cc


#############################################

def col_con3(df, index): # C/O\nAC
  time = []
  ac = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      time.append(item)
      ac.append("")

    elif len(item) == 0:
      time.append("")
      ac.append("")

    elif len(item) == 3:
      time.append("")
      ac.append(item)

    else:    
      item = item.split()
      for items in item:
        #print(items)
        if len(items) == 5:
          time.append(items)
          #dd.append("")
          #print("entered")
        else:
          #new = date[-1]
          #date.append(new)
          ac.append(items)
          #print("n entered")

  #print(col_one_list)
  #print(len(time))
  #print(time)
  #print(len(ac))
  #print(ac) 

  return time, ac


#############################################

def col_con4(df, index): # ATA\nC/O
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return ata, co

#############################################
global final_date

def col_con5(df, index): # Date
  global final_date
  date = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      date.append(item)
      
    else:
      try:
        new = date[-1]
        date.append(new)
      except:
        date.append(final_date)
        

  #print(col_one_list)
  #print(len(date))
  #print(date)
  
  final_date = date[-1]

  return date



#############################################

def col_con6(df, index): # BLHCR\nFDUTY
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 5:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return co, ata 



#############################################

def col_con7(df, index): # AC\nRq
  ata = []
  co = []
  mix = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    if len(item) == 3:
      ata.append(item)
      co.append("")

    elif len(item) == 0:
      ata.append("")
      co.append("")

    else:    
      item = item.split()
      ata.append(item[0])
      co.append(item[1])
      

  #print(col_one_list)
  #print(len(ata))
  #print(ata)
  #print(len(co))
  #print(co) 

  return co, ata 



#############################################

def col_con8(df, index): # C/I\nDate\nDD\nActivity\nOrig
  global final_date
  
  cc = 'ASP ABH ARY ARM AVV AYQ BSJ BNK BZD BCI BHS BRT BEU BLN BKQ BLT BQL BRK BWQ BNE BHQ BME BDB CNS CDU CBR CVQ CED CTL CCL CMQ CVC CNJ CAZ CUQ CFS CPD CTN OOM COJ CNB CMD CWW CYG CWT CMA DRW DNQ DRB DPO DRN DBO DYA ECH ELC EMD EPR FIZ FRB FOS GPN GET GLT GLI OOL GUL GOV GFN GFF GTE GUH HCQ HLT HTI HXX HVB HBA HTU HID HSM IVR KGI KTA KPS KRA KNS KGY KGC KWM KNX TGN LST LEA LGH LER LNO LHG LSY IRG LRE MKY MNG MRG MKR MEB MEL MBW MIM MQL MOV MRZ MYA MGB MHU ISA WME MMG DGE NAA NRA QRM NTL ZNE RPM NTN NOA OKY OAG PBO PKE PER JAD PUG PHE PKT PLO PQQ PPI PTJ PPP ULP YAMB DCN YMES XRH RMK ROK RMA MJK SHT SIO SNB SWC MCY SWH BWU SYD TMW TRO TEF TEM TCA THG XTG TYB KTR WTB TSV TUM WGA WGE WGT WKB QRR WEI SXE WWY WYA WUN WNR WIN WOL BWT NGA'
  city_codes = cc.split(' ')
  
  ci = []
  date = []
  dd = []  
  activity = []
  orig = []

  ci_n = ""
  date_n = ""
  dd_n = ""
  activity_n = ""
  orig_n = ""


  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    item = item.split()

    for items in item:
      if len(items) == 5:
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          date_n = items
          
      if (len(items) == 5) and (":" in items):
        ci_n = items

      if items in city_codes:    
        orig_n = items        

      if (len(items) >= 3) and (len(items) <= 5) and (":" not in items) and (items not in city_codes):
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          a = 0
        else:
          activity_n = items
          #dd_n = ""

      if (len(items) < 3):
        dd_n = items
    
    #if len(items) == 5:
    date.append(date_n)
    ci.append(ci_n)
    orig.append(orig_n)
    activity.append(activity_n)
    try:
      dd.append(dd_n)
    except:
      dd.append("")


    ci_n = ""
    dd_n = ""
    activity_n = ""
    orig_n = ""


  #print(ci)
  #print(len(ci))
  #print(date)
  #print(len(date))
  #print(dd)
  #print(len(dd))
  #print(activity)
  #print(len(activity))
  #print(orig)
  #print(len(orig))

  return ci, date, dd, activity, orig


#############################################

def col_con9(df, index): # C/I\nDate\nDD\nActivity
  global final_date

  ci = []
  date = []
  dd = []  
  activity = []

  ci_n = ""
  date_n = ""
  dd_n = ""
  activity_n = ""


  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    #item = item.replace('\n', ' ')
    item = item.split()

    for items in item:
      if len(items) == 5:
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          date_n = items
          
      if (len(items) == 5) and (":" in items):
        ci_n = items
      

      if (len(items) >= 3) and (len(items) <= 5) and (":" not in items):
        day = items[-3:]
        if (day == "MON") or (day == "TUE") or (day == "WED") or (day == "THU") or (day == "FRI") or (day == "SAT") or (day == "SUN"):
          a = 0
        else:
          activity_n = items
          #dd_n = ""

      if (len(items) < 3):
        dd_n = items
        
    #if len(items) == 5:
    date.append(date_n)
    ci.append(ci_n)
    activity.append(activity_n)
    try:
      dd.append(dd_n)
    except:
      dd.append("")


    ci_n = ""
    dd_n = ""
    activity_n = ""


  #print(ci)
  #print(len(ci))
  #print(date)
  #print(len(date))
  #print(dd)
  #print(len(dd))
  #print(activity)
  #print(len(activity))
  #print(orig)
  #print(len(orig))

  return ci, date, dd, activity


#############################################

def col_con10(df, index): # C/O\nAC\nRq
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:

    if (len(item) < 5) and (len(item) > 0):
      time.append("")
      cc.append(item)

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5) and (":" in item):
      time.append(item)
      cc.append("")

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if len(items) == 3:    
          cc.append(items)        

        if (":" in items):
          time.append(items)


  return time, cc


#############################################

def col_con11(df, index): # Orig
  ccs = 'ASP ABH ARY ARM AVV AYQ BSJ BNK BZD BCI BHS BRT BEU BLN BKQ BLT BQL BRK BWQ BNE BHQ BME BDB CNS CDU CBR CVQ CED CTL CCL CMQ CVC CNJ CAZ CUQ CFS CPD CTN OOM COJ CNB CMD CWW CYG CWT CMA DRW DNQ DRB DPO DRN DBO DYA ECH ELC EMD EPR FIZ FRB FOS GPN GET GLT GLI OOL GUL GOV GFN GFF GTE GUH HKT HCQ HLT HTI HXX HVB HBA HTU HID HSM IVR KGI KTA KPS KRA KNS KGY KGC KWM KNX TGN LST LEA LGH LER LNO LHG LSY IRG LRE MKY MNG MRG MKR MEB MEL MBW MIM MQL MOV MRZ MYA MGB MHU ISA WME MMG DGE NAA NRA QRM NTL ZNE RPM NTN NOA OKY OAG PBO PKE PER JAD PUG PHE PKT PLO PQQ PPI PTJ PPP ULP YAMB DCN YMES XRH RMK ROK RMA MJK SHT SIO SNB SWC MCY SWH BWU SYD TMW TRO TEF TEM TCA THG XTG TYB KTR WTB TSV TUM WGA WGE WGT WKB QRR WEI SXE WWY WYA WUN WNR WIN WOL BWT NGA'
  city_codes = ccs.split(' ')
  
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
  
    if (len(item) == 3):
      time.append(item)
      cc.append("")

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5):
      time.append("")
      cc.append(item)

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if items in city_codes:    
          time.append(items)        

        if (":" in items):
          cc.append(items)


  return time, cc



#############################################

def col_con12(df, index): # C/O\nAC\nRq
  time = []
  cc = []

  col_one_list = df[index].tolist()
  #print(len(col_one_list))
  #print(col_one_list)

  res = []
  for sub in col_one_list:
    res.append(sub.replace("\n", " "))

  for item in col_one_list:
    
    if (len(item) < 5) and (len(item) > 0):
      time.append("")
      cc.append(item)

    if (len(item) == 0):
      time.append("")
      cc.append("")

    if (len(item) == 5) and (":" in item):
      time.append(item)
      cc.append("")

    if (len(item) > 5):
      item = item.split()

      for items in item:
        
        if len(items) == 3:    
          cc.append(items)        

        if (":" in items):
          time.append(items)



  return time, cc

################################################################################################################################################################


def Final(df):
  df1 = df
  column_headers = df.columns.values.tolist()
  #print(column_headers)
  #print()

  cols_index = []

  i = 0
  for columns in column_headers:
    if columns == "Date\nDD":
      index = i
      date, dd = col_con1(df1, column_headers[i])
      
      cols_index.append(index)    
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Date", date)
      df1.insert(((index+1)+1), "Duty Designator", dd)
    
    if columns == "C/I\nActivity":
      index = i
      date, dd = col_con2(df1, (column_headers[i+1]))

      df1.rename(columns = {'C/I\nActivity':'Activity'}, inplace = True)
      cols_index.append(index)
      columns = column_headers[index+1]
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", date)
      df1.insert(((index+1)+1), "Origin", dd)
      
    if columns == "C/I\nOrig":
      index = i
      date, dd = col_con2(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", date)
      df1.insert(((index+1)+1), "Origin", dd)
    
    if columns == "ATA\nC/O":
      index = i
      date, dd = col_con3(df1, (column_headers[i+1]))
      ata, co = col_con4(df1, (column_headers[i]))

      cols_index.append(index)
      columns1 = column_headers[index+1]
      df1.drop(columns1, axis=1, inplace=True)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)
      
      df1.insert(((index+1)+2), "ATA", ata)
      df1.insert(((index+1)+3), "C O", co)
    
      df1["Check Out Time"] = df1["Check Out Time"] + df1["C O"]
      df1.drop('C O', axis=1, inplace=True)


    if columns == "STA\nC/O":
      index = i
      date, dd = col_con3(df1, (column_headers[i+1]))
      ata, co = col_con4(df1, (column_headers[i]))

      cols_index.append(index)
      columns1 = column_headers[index+1]
      df1.drop(columns1, axis=1, inplace=True)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)
      
      df1.insert(((index+1)+2), "STA", ata)
      df1.insert(((index+1)+3), "C O", co)
    
      df1["Check Out Time"] = df1["Check Out Time"] + df1["C O"]
      df1.drop('C O', axis=1, inplace=True)



    if columns == "BLHCR\nFDUTY":
      index = i
      blhcr, fduty = col_con6(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      #df1.insert((index+1), "BLHCR", blhcr)
      #df1.insert(((index+1)+1), "FDUTY", fduty)

    if columns == "AC\nRq":
      index = i
      blhcr, fduty = col_con7(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "AC", fduty)
      df1.insert(((index+1)+1), "Rq", blhcr)

    if columns == "C/I\nDate\nDD\nActivity\nOrig":
      index = i
      ci, date, dd, activity, orig = col_con8(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", ci)
      df1.insert(((index+1)+1), "Date", date)
      df1.insert(((index+1)+1), "DD", dd)
      df1.insert(((index+1)+1), "Activity", activity)
      df1.insert(((index+1)+1), "Orig", orig)

    if columns == "C/I\nDate\nDD\nActivity":
      index = i
      ci, date, dd, activity = col_con9(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "Check In Time", ci)
      df1.insert(((index+1)+1), "Date", date)
      df1.insert(((index+1)+1), "DD", dd)
      df1.insert(((index+1)+1), "Activity", activity)

    if columns == "C/O\nAC\nRq":
      index = i
      date, dd = col_con10(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "Check Out Time", date)
      df1.insert(((index+1)+1), "Aircraft Type", dd)

    if (columns == "Orig"):
      index = i
      try:
        date, dd = col_con11(df1, (column_headers[i]))


        cols_index.append(index)
        df1.drop(columns, axis=1, inplace=True)
        
        df1.insert((index+1), "Origin", date)
        df1.insert(((index+1)+1), "C I", dd)

        df1["Check In Time"] = df1["Check In Time"] + df1["C I"]
        df1.drop('C I', axis=1, inplace=True)

      except:
        pass

    if columns == "ATA\nDest":
      index = i
      date, dd = col_con12(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "ATA", date)
      df1.insert(((index+1)+1), "Dest", dd)

    if columns == "STA\nDest":
      index = i
      date, dd = col_con12(df1, (column_headers[i]))

      cols_index.append(index)
      df1.drop(columns, axis=1, inplace=True)
      
      df1.insert((index+1), "STA", date)
      df1.insert(((index+1)+1), "Dest", dd)

    if columns == "Date":
      index = i
      date = col_con5(df1, column_headers[i])
      
      cols_index.append(index)    
      df1.drop(columns, axis=1, inplace=True)
      df1.insert((index+1), "nDate", date)
    
      df1["Date"] = df1["nDate"]
      df1.drop("nDate", axis=1, inplace=True)


    i += 1

  try:
    df1.drop("BLH", axis=1, inplace=True)
    df1.drop("Rq", axis=1, inplace=True)
    df1.drop("BLHCR", axis=1, inplace=True)
    df1.drop("FDUTY", axis=1, inplace=True)
  except:
    pass

  return df1



################################################################################################################################################################



def Last(pdf, df):

  global_month = ''
  global_year = ''
  max = 0

  global_month, global_year, max, home_base = getter(pdf)

  #df = df

  df['WDO'] = 'FALSE'

  arr = []

  #print(df)
  try:
    df.loc[df['Activity'].str.startswith('WDO'), 'WDO'] = 'TRUE'
  except:
    df2=df.dropna()
    #print(df2)

  lister = df.to_dict('dict')


  data = lister['WDO']


  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows = []
  value = 'TRUE'
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Date']

  rows1 = []
  dates = []

  for item in rows:  
    value_at_index = list(data2.values())[item]
    #print(value_at_index)
    dates.append(value_at_index)


  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows1.append(key)
              do_exist = True
      return do_exist


  for value in dates:
    # Iterate over all key, value pairs in dict and check if value exist
    if check_value_exist(data2, value, rows1):
        o = 0
        #print(f"Yes, Value: '{value}' exists in dictionary")
    else:
        o = 0
        #print(f"No, Value: '{value}' does not exists in dictionary")

  def checkKey(data, row):
    data[row] = 'TRUE'  
    #lister['WDO'][row].update('TRUE')
    #lister['WDO'][row].pop(1)
    #print(data)
          
  for row in rows1:
    checkKey(data, row)

  lister['WDO'] = data

  new = pd.DataFrame.from_dict(lister)
  #print(new)
  #new = new.iloc[: , 1:]
  #new['WDO'] = new['WDO'] 
  #new['WDO?'] = new.loc[:, 'WDO']

  def date_cols(df):
    date = []
    day = []

    col_one_list = df['Date'].tolist()
    
    res = []

    for item in col_one_list:
      big = item
      small = item
      date1 = big[:-3]
      day1 = small[2:]
      date.append(date1)
      day.append(day1)        

    

    #print(date)
    #print(len(date))
    #print()
    #print(dd)
    #print(len(dd))

    #print(col_one_list)
    
    return date, day
    
  
  column_headers1 = list(new.columns)
  #print(column_headers1)
  if ("Aircraft Type" in column_headers1) and ("AC" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Aircraft Type"] = new["AC"] + '' + new["Aircraft Type"]
    new.drop('AC', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Duty Designator" in column_headers1) and ("DD" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Duty Designator"] = new["DD"] + '' + new["Duty Designator"]
    new.drop('DD', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Check Out Time" in column_headers1) and ("C/O" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Check Out Time"] = new["C/O"] + '' + new["Check Out Time"]
    new.drop('C/O', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Check In Time" in column_headers1) and ("C/I" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Check In Time"] = new["C/I"] + '' + new["Check In Time"]
    new.drop('C/I', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0

  if ("Orig" in column_headers1) and ("Origin" in column_headers1):
    new.fillna(' ', inplace=True)
    new["Origin"] = new["Orig"] + '' + new["Origin"]
    new.drop('Orig', axis=1, inplace=True)
    #print("entered and done")
  else:
    a = 0


  column_headers = list(new.columns)
  #print(column_headers)

  # Activity', 'Date', 'Duty Designator', 'C/I', 'Orig', 'ATD', 'Dest', 'ATA', 'C/O', 'AC', 'DD', 'WDO
  for names in column_headers:
    if names == "C/I":
      new.rename(columns = {'C/I':'Check In Time'}, inplace = True)
    elif names == "Orig":
      new.rename(columns = {'Orig':'Origin'}, inplace = True)
    elif names == "C/O":
      new.rename(columns = {'C/O':'Check Out Time'}, inplace = True)
    elif names == "AC":
      new.rename(columns = {'AC':'Aircraft Type'}, inplace = True)    
    elif names == "DD":
        new.rename(columns = {'DD':'Duty Designator'}, inplace = True)
        
  #print(new)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'ATA', 'Check Out', 'Date  ==>   better names
    dataframe = new[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Origin', 'WDO', 'ATD', 'Dest', 'ATA', 'Check Out Time', 'Date']]
  except:
    dataframe = new[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Origin', 'WDO', 'STD', 'Dest', 'STA', 'Check Out Time', 'Date']]


  date = []
  day = []

  date, day = date_cols(dataframe)

  dataframe.rename(columns = {'Date':'full date'}, inplace = True)
  #print()
  #print("FINAL")

  dataframe.insert(1, "Date", date)
  dataframe.insert(2, "Day", day)
  dataframe.insert(3, "Month", global_month)
  dataframe.insert(4, "Year", global_year)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Destination', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out', 'Date', 'Day', 'Month', 'Year'
    #dataframe1 = dataframe[['Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'WDO', 'ATA', 'Check Out']]
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']]
  except:
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'STD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'STA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']]


  #print(dataframe)  

  return dataframe

#################################################################################



def Neat(dataframe):

  dataframe['Origin'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['Origin'].transform(lambda x: ''.join(x))
  dataframe['Dest'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['Dest'].transform(lambda x: ''.join(x))

  try:
    dataframe['ATA'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['ATA'].transform(lambda x: ''.join(x))
    dataframe['ATD'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['ATD'].transform(lambda x: ''.join(x))
  except:
    dataframe['STA'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['STA'].transform(lambda x: ''.join(x))
    dataframe['STD'] = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out Time', 'Date', 'Day', 'Month', 'Year']].groupby(['Activity','Aircraft Type'])['STD'].transform(lambda x: ''.join(x))  


  def add_one(x):
    #print(x)
    x = str(x)
    if len(x) > 3:
      x = x[:3]
    else:
      x = x
    
    return x

  def add_two(x):
    #print(x)
    x = str(x)
    if len(x) > 5:
      x = x[:5]
    else:
      x = x
    
    return x


  dataframe['Origin'] = dataframe['Origin'].apply(add_one)
  dataframe['Dest'] = dataframe['Origin'].apply(add_one)

  try:
    dataframe['ATA'] = dataframe['ATA'].apply(add_two)
    dataframe['ATD'] = dataframe['ATD'].apply(add_two)
  except:  
    dataframe['STA'] = dataframe['STA'].apply(add_two)
    dataframe['STD'] = dataframe['STD'].apply(add_two)



################################################################################################################################################################


def lines(pdf, df):

  global_month = ''
  global_year = ''
  max = 0

  global_month, global_year, max, home_base = getter(pdf)

  lister = df.to_dict('dict')


  data = lister['Dest']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Dest']

  dates = []

  for item in rows1:
    value_at_index = list(data2.values())[item+1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  #print("data from dest    ", data)
  lister['Dest'].update(data)

  try:
    data = lister['ATA']
  except:
    data = lister['STA']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  try:
    data2 = lister['ATA']
  except:
    data2 = lister['STA']

  dates = []

  for item in rows1:  
    value_at_index = list(data2.values())[item+1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  
  try:
    #print("data from ata    ", data)
    lister['ATA'].update(data)
  except:
    #print("data from sta    ", data)
    lister['STA'].update(data)

  save1 = rows1

  ##########################################################################################################################################################
  

  data = lister['Origin']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  data2 = lister['Origin']

  dates = []

  for item in rows1:
    value_at_index = list(data2.values())[item-1]
    #print(value_at_index)
    dates.append(value_at_index)

  #print(dates)

  def checkKey(data, row, dates):
    data[row] = dates

  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  #print("data from origin    ", data)
  lister['Origin'].update(data)


  save2 = rows1

  try:
    data = lister['ATD']
  except:
    data = lister['STD']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val == value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  try:
    data2 = lister['ATD']
  except:
    data2 = lister['STD']

  dates = []

  _o0 = lister['Origin']
  try:
    _o1 = lister['ATD']
  except:
    _o1 = lister['STD']

  _o2 = lister['Dest']
  try:
    _o3 = lister['ATA']
  except:
    _o3 = lister['STA']

  for item in rows1:  
    if (list(_o0.values())[item] == "") and (list(_o1.values())[item] == ""):
      value_at_index = list(data2.values())[item-1]
      #print(value_at_index)
      dates.append(value_at_index)
    else:
      value_at_index = list(data2.values())[item+1]
      #print(value_at_index)
      dates.append(value_at_index)


  def checkKey(data, row, dates):
    data[row] = dates
  
  i = 0
  for row in rows1:
    checkKey(data, row, dates[i])
    i += 1

  try:
    #print("data from atd    ", data)
    lister['ATD'].update(data)
  except:
    #print("data from std    ", data)
    lister['STD'].update(data)

  #print(lister)

  #save2 = rows1

  ##########################################################################################################################################################
  
  new = []
  lol = ''
  i = 0 
  for dates in save1:
    lol = str(save1[i]) + "," + str(save2[i]) 
    i += 1
    new.append(lol)

  #print(new)
  #print()

  #######################
  
  data1 = lister['Date_3']
  data2 = lister['Date_4']
  
  def checkKey(data, row, dates):
    data[row] = dates

  for some in new:
    one, two = some.split(',')
    one = int(one)
    two = int(two)
    #print(one)
    #print(two)

    checkKey(data1, one, data1[two+1])
    checkKey(data2, one, data2[two+1])
    

  #######################
  

  data1 = lister['Day_3']
  data2 = lister['Day_4']
  data3 = lister['Check Out Time']
  
  def checkKey(data, row, dates):
    data[row] = dates
    if dates != "":
      #print("finally-------------------------------------------------------------------------------------")
      data[row] = str(dates)
    
  for some in new:
    one, two = some.split(',')
    one = int(one)
    two = int(two)
    #print(one)
    #print(two)
    checkKey(data1, one, data1[two+1])
    checkKey(data2, one, data2[two+1])
    checkKey(data3, one, data3[two])
    
  ##########################################################################################################################################################

  dataframe = pd.DataFrame.from_dict(lister)

  #print(save2)
  #print()
  for i in save2:
    #print(i)
    dataframe = dataframe.drop(i)

  dataframe = dataframe.reset_index(drop=True)

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Destination', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out', 'Date', 'Day', 'Month', 'Year'
    #dataframe1 = dataframe[['Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'WDO', 'ATA', 'Check Out']]
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date_2', 'Day_2', 'Month', 'Year', 'Dest', 'WDO', 'ATA', 'Date_3', 'Day_3', 'Month', 'Year', 'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]
  except:
    dataframe = dataframe[['Activity', 'Duty Designator', 'Aircraft Type', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Origin', 'WDO', 'STD', 'Date_2', 'Day_2', 'Month', 'Year', 'Dest', 'WDO', 'STA', 'Date_3', 'Day_3', 'Month', 'Year', 'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]


  #print(dataframe)  

  return dataframe

#################################################################################






################################################################################################################################################################

import PyPDF2


### DRIVER CODE


### FILE PATH
##################################                          INSERT FILE NAME UNDER HERE ↓↓↓↓                          ##################################
pdf = 'Sep Actual Roster.pdf'
##################################                          INSERT FILE NAME UNDER HERE ↑↑↑↑                          ##################################

pages = Checker(pdf)
print("total pages: ", (pages+1))

appended_df = []

#for i in range(1):
for i in range((pages+1)):
  #print(i)
  #i = 1
  df = Main(pdf, i)
  df = pd.DataFrame(df)
  df = df.rename(columns=df.iloc[1]).loc[2:]
  df = Final(df)
  appended_df.append(df)


import pandas as pd
try:
  df = pd.concat(appended_df, ignore_index=True)
except:
  new_appended_df = appended_df[:-1]
  df = pd.concat(new_appended_df, ignore_index=True)

#df

df = df.reset_index(drop=True)

dataframe = Last(pdf, df)
dataframe = dataframe.fillna('')


#Neat(dataframe)

cols = []
count = 1
for column in dataframe.columns:
    if column == 'Date':
        cols.append(f'Date_{count}')
        count+=1
        continue
    cols.append(column)
dataframe.columns = cols


cols = []
count = 1
for column in dataframe.columns:
    if column == 'Day':
        cols.append(f'Day_{count}')
        count+=1
        continue
    cols.append(column)
dataframe.columns = cols


dataframe = lines(pdf, dataframe)

print()
dataframe

total pages:  2


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1348: UserWarning: DataFrame columns are not unique, some columns will be omitted.


,Activity,Duty Designator,Aircraft Type,Check In Time,Date_1,Day_1,Month,Year,Origin,WDO,...,ATA,Date_3,Day_3,Month,Year,Check Out Time,Date_4,Day_4,Month,Year
0,OFF,,,,31,WED,SEP,22,MEL,FALSE,...,24:00,31,WED,SEP,22,,31,WED,SEP,22
1,OFF,,,,01,THU,SEP,22,MEL,FALSE,...,24:00,01,THU,SEP,22,,01,THU,SEP,22
2,CO2,,,08:30,02,FRI,SEP,22,MEL,FALSE,...,11:35,02,FRI,SEP,22,,02,FRI,SEP,22
3,JQ796,,320,11:45,02,FRI,SEP,22,MEL,FALSE,...,15:03,02,FRI,SEP,22,,02,FRI,SEP,22
4,JQ797,,320,,02,FRI,SEP,22,MCY,FALSE,...,18:11,02,FRI,SEP,22,18:26,02,FRI,SEP,22
5,JQ518,,321,14:40,03,SAT,SEP,22,MEL,FALSE,...,17:17,03,SAT,SEP,22,,03,SAT,SEP,22
6,JQ525,,321,,03,SAT,SEP,22,SYD,FALSE,...,19:30,03,SAT,SEP,22,,03,SAT,SEP,22
7,JQ532,,321,,03,SAT,SEP,22,MEL,FALSE,...,21:40,03,SAT,SEP,22,21:55,03,SAT,SEP,22
8,PCK,,,,03,SAT,SEP,22,SYD,FALSE,...,22:25,03,SAT,SEP,22,,03,SAT,SEP,22
9,HTL,,,,03,SAT,SEP,22,SYD,FALSE,...,19:05,04,SUN,SEP,22,,04,SUN,SEP,22


In [ ]:
def duties(pdf, df):

  global_month = ''
  global_year = ''
  max1 = 0

  global_month, global_year, max1, home_base = getter(pdf)


  lister = df.to_dict('dict')



  data = lister['Check In Time']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if val != value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  save1 = rows1
  print(save1)

  ##########################################################################################################################################################


  data = lister['Check Out Time']

  def check_value_exist(test_dict, value, rows):
      do_exist = False
      for key, val in test_dict.items():
          if str(val) != value:
              #print(key)
              rows.append(key)
              do_exist = True
      return do_exist


  rows1 = []
  value = ''
  # Iterate over all key, value pairs in dict and check if value exist
  if check_value_exist(data, value, rows1):
    o = 0
      #print(f"Yes, Value: '{value}' exists in dictionary")
  else:
    o = 0
    #print(f"No, Value: '{value}' does not exists in dictionary")

  save2 = rows1
  print(save2)

  ##########################################################################################################################################################


  #######################
  

  save1_size = len(save1)
  save2_size = len(save2)

  larger = max(save1_size, save2_size)
  print("this is the max length of loop: ", larger)


  def check_nxt(current1, current2, next1):
    if current2 >= current1:  # 4 > 2 T
      if current2 >= next1:   # 4 > 3 T
        current1 = next1    # 2 ==> 3

      else:
        a = 0

    else:
      current2 = next2    # 

    return current1, current2, next1

  end = ''
  for i in range(larger):
    try:
      current1 = save1[i]   #   [3, 5, 11, 15, 25, 28, 33, 39, 42, 43, 50, 52, 54]   3
    except:
      continue

    try:
      current2 = save2[i]   #   [4, 7, 11, 16, 22, 26, 28, 32, 33, 40, 46, 51, 53, 55]  4
    except:
      continue


    try:
      next1 = save1[i+1]    #   [2, 3, 5, 11, 15, 25, 28, 33, 39, 42, 43, 50, 52, 54]   3
    except:
      end == "max"
      next1 = save1[i]
      continue

    try:
      next2 = save2[i+1]    #   [4, 7, 11, 16, 22, 26, 28, 32, 33, 40, 46, 51, 53, 55]  7
    except:
      end == "max"
      next2 = save2[i]

    current1, current2, next1 = check_nxt(current1, current2, next1)

    if end == "max":
      break

    # 3,4   5,7   11,11   15,16   25,22


  #######################
  #######################
  #######################
  
















  ##########################################################################################################################################################

  dataframe = pd.DataFrame.from_dict(lister)

  

  try:
    #   Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Date', 'Day', 'Month', 'Year', 'Origin', 'WDO', 'ATD', 'Date', 'Day', 'Month', 'Year', 'Destination', 'WDO', 'ATA', 'Date', 'Day', 'Month', 'Year', 'Check Out', 'Date', 'Day', 'Month', 'Year'
    #dataframe1 = dataframe[['Activity', 'Duty Designator', 'Aircraft', 'Check In', 'Origin', 'WDO', 'ATD', 'Destination', 'WDO', 'ATA', 'Check Out']]
    dataframe = dataframe[['Activity', 'Origin', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Dest',  'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]
  except:
    dataframe = dataframe[['Activity', 'Origin', 'Check In Time', 'Date_1', 'Day_1', 'Month', 'Year', 'Dest',  'Check Out Time', 'Date_4', 'Day_4', 'Month', 'Year']]


  #print(dataframe)  

  return dataframe

#################################################################################


dataframe = duties(pdf, dataframe)

print()
dataframe

[2, 3, 5, 11, 15, 20, 25, 28, 32, 33, 39, 42, 43, 50, 52, 54]
[4, 7, 11, 16, 22, 26, 28, 32, 33, 40, 46, 51, 53, 55]
this is the max length of loop:  16



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  # Remove the CWD from sys.path while we load stuff.


,Activity,Origin,Check In Time,Date_1,Day_1,Month,Year,Dest,Check Out Time,Date_4,Day_4,Month,Year
0,OFF,MEL,,31,WED,SEP,22,MEL,,31,WED,SEP,22
1,OFF,MEL,,01,THU,SEP,22,MEL,,01,THU,SEP,22
2,CO2,MEL,08:30,02,FRI,SEP,22,MEL,,02,FRI,SEP,22
3,JQ796,MEL,11:45,02,FRI,SEP,22,MCY,,02,FRI,SEP,22
4,JQ797,MCY,,02,FRI,SEP,22,MEL,18:26,02,FRI,SEP,22
5,JQ518,MEL,14:40,03,SAT,SEP,22,SYD,,03,SAT,SEP,22
6,JQ525,SYD,,03,SAT,SEP,22,MEL,,03,SAT,SEP,22
7,JQ532,MEL,,03,SAT,SEP,22,SYD,21:55,03,SAT,SEP,22
8,PCK,SYD,,03,SAT,SEP,22,SYD,,03,SAT,SEP,22
9,HTL,SYD,,03,SAT,SEP,22,SYD,,04,SUN,SEP,22


In [ ]:
import pandas as pd

CI = []
CO = []
all_time = []

for index, row in dataframe.iterrows():
  index_s = str(index)
  if len(index_s) == 1:
    index_s = "0" + str(index)

  CI_cur = index_s + "__" + row['Check In Time'] + "__ci"
  CO_cur = index_s + "__" + row['Check Out Time'] + "__co"



  if (len(CI_cur) <= 9):
    a = 0
  else:
    all_time.append(CI_cur)
  
  if (len(CO_cur) <= 9):
    a = 0
  else:
    all_time.append(CO_cur)
  

#print(all_time)

final_time = []



from itertools import tee, islice, chain

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)


for previous, item, nxt in previous_and_next(all_time):
  current = item
  next = nxt
  #prev = previous

  #print(prev)
  #print(current)
  #print(next)


  if ("_ci" in current) and ("_co" in next):
    final_time.append(current)
    final_time.append(next)





final_time
#all_time

['03__11:45__ci',
 '04__18:26__co',
 '05__14:40__ci',
 '07__21:55__co',
 '11__19:35__ci',
 '11__01:53__co',
 '15__00:30__ci',
 '16__09:51__co',
 '20__22:40__ci',
 '22__16:25__co',
 '25__16:15__ci',
 '26__20:58__co',
 '28__16:30__ci',
 '28__22:36__co',
 '32__22:00__ci',
 '32__07:55__co',
 '33__10:30__ci',
 '33__19:00__co',
 '39__14:10__ci',
 '40__22:59__co',
 '43__06:30__ci',
 '46__17:57__co',
 '50__14:10__ci',
 '51__23:13__co',
 '52__13:55__ci',
 '53__00:42__co',
 '54__13:55__ci',
 '55__00:26__co']

In [ ]:
initial = []
final = []

for time in final_time:
  #print(time)
  if time[-2:] == "ci":
    word = time[:3]
    word = word.replace("_","")
    initial.append(word)
  
  if time[-2:] == "co":
    word = time[:3]
    word = word.replace("_","")
    final.append(word)


combined_index = initial + final  
combined_index.sort()  
print("Sorted list is:",combined_index)  

df1 = dataframe.iloc[initial]
df1 = df1.sort_index()
df1 = df1.drop('Dest', axis=1)
df1 = df1.drop('Check Out Time', axis=1)
df1 = df1.drop('Date_4', axis=1)
df1 = df1.drop('Day_4', axis=1)
df1

df2 = dataframe.iloc[final]
df2 = df2.sort_index()
df2 = df2.drop('Origin', axis=1)
df2 = df2.drop('Check In Time', axis=1)
df2 = df2.drop('Date_1', axis=1)
df2 = df2.drop('Day_1', axis=1)
df2 = df2.drop('Month', axis=1)
df2 = df2.drop('Year', axis=1)
df2


Sorted list is: ['03', '04', '05', '07', '11', '11', '15', '16', '20', '22', '25', '26', '28', '28', '32', '32', '33', '33', '39', '40', '43', '46', '50', '51', '52', '53', '54', '55']


,Activity,Dest,Check Out Time,Date_4,Day_4
4,JQ797,MEL,18:26,02,FRI
7,JQ532,SYD,21:55,03,SAT
11,JQ672,DRW,01:53,05,MON
16,JQ563,MEL,09:51,06,TUE
22,QF156,MEL,16:25,10,SAT
26,JQ779,MEL,20:58,11,SUN
28,JQ035,DPS,22:36,12,MON
32,JQ1036,MEL,07:55,15,THU
33,EPT,MEL,19:00,15,THU
40,JQ945,MEL,22:59,21,WED


In [ ]:
lister = df1.to_dict('dict')


data = lister['Check In Time']

def check_value_exist(test_dict, value, rows):
    do_exist = False
    for key, val in test_dict.items():
        if val == value:
            #print(key)
            rows.append(key)
            do_exist = True
    return do_exist


rows1 = []
value = ''
# Iterate over all key, value pairs in dict and check if value exist
if check_value_exist(data, value, rows1):
  o = 0
    #print(f"Yes, Value: '{value}' exists in dictionary")
else:
  o = 0
  #print(f"No, Value: '{value}' does not exists in dictionary")

data2 = lister['Check In Time']

dates = []

for item in rows1:
  try:
    value_at_index = list(data2.values())[item-1]
    #print(value_at_index)
    dates.append(value_at_index)
  except:
    pass

#print(dates)

def checkKey(data, row, dates):
  data[row] = dates

i = 0
for row in rows1:
  try:
    checkKey(data, row, dates[i])
  except:
    pass
  i += 1

#print("data from origin    ", data)
lister['Check In Time'].update(data)


save1 = rows1
save1




data = lister['Check Out Time']

def check_value_exist(test_dict, value, rows):
    do_exist = False
    for key, val in test_dict.items():
        if val == value:
            #print(key)
            rows.append(key)
            do_exist = True
    return do_exist


rows1 = []
value = ''
# Iterate over all key, value pairs in dict and check if value exist
if check_value_exist(data, value, rows1):
  o = 0
    #print(f"Yes, Value: '{value}' exists in dictionary")
else:
  o = 0
  #print(f"No, Value: '{value}' does not exists in dictionary")

data2 = lister['Check Out Time']

dates = []

for item in rows1:
  try:
    value_at_index = list(data2.values())[item-1]
    #print(value_at_index)
    dates.append(value_at_index)
  except:
    pass

#print(dates)

def checkKey(data, row, dates):
  data[row] = dates

i = 0
for row in rows1:
  try:
    checkKey(data, row, dates[i])
  except:
    pass
  i += 1

#print("data from origin    ", data)
lister['Check Out Time'].update(data)


save2 = rows1
save2







new = []
lol = ''
i = 0 
for dates in save1:
  lol = str(save1[i]) + ":" + str(save2[i]) 
  i += 1
  new.append(lol)

#print(new)
#print()

#######################

data2 = lister['Date_4']

def checkKey(data, row, dates):
  data[row] = dates

for some in new:
  one, two = some.split(':')
  one = int(one)
  two = int(two)
  #print(one)
  #print(two)
  try:
    checkKey(data2, one, data2[two+1])
  except:
    pass

#######################


data2 = lister['Day_4']
data3 = lister['Check Out Time']

def checkKey(data, row, dates):
  data[row] = dates
  if dates != "":
    #print("finally-------------------------------------------------------------------------------------")
    data[row] = str(dates)
  
for some in new:
  one, two = some.split(':')
  one = int(one)
  two = int(two)
  #print(one)
  #print(two)
  try:
    checkKey(data2, one, data2[two+1])
  except:
    pass
  try:
    checkKey(data3, one, data3[two])
  except:
    pass
  
##########################################################################################################################################################

dataframe = pd.DataFrame.from_dict(lister)
dataframe

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


,Activity,Origin,Check In Time,Date_1,Day_1,Month,Year,Dest,Check Out Time,Date_4,Day_4
3,JQ796,MEL,11:45,02,FRI,SEP,22,MCY,,02,FRI
4,JQ797,MCY,,02,FRI,SEP,22,MEL,,02,FRI
5,JQ518,MEL,14:40,03,SAT,SEP,22,SYD,01:53,03,SAT
7,JQ532,MEL,,03,SAT,SEP,22,SYD,01:53,03,SAT
11,JQ672,SYD,19:35,04,SUN,SEP,22,DRW,01:53,05,MON
15,JQ673,DRW,00:30,06,TUE,SEP,22,BNE,,06,TUE
16,JQ563,BNE,,06,TUE,SEP,22,MEL,,06,TUE
20,JQ173,MEL,22:40,09,FRI,SEP,22,CHC,23:13,10,SAT
22,QF156,AKL,,10,SAT,SEP,22,MEL,23:13,10,SAT
25,JQ780,MEL,16:15,11,SUN,SEP,22,ADL,,11,SUN
